In [ ]:
# Copyright 2021 Google
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
!pip install cirq --pre

In [2]:
import cirq
import itertools
import numpy as np
import matplotlib.pyplot as plt
import recirq.time_crystals as time_crystals

In [3]:
# define the qubits we'll be using
qubit_locations = [(3, 9), (3, 8), (3, 7), (4, 7), (4, 8), (5, 8), (5, 7), (5, 6), (6, 6), (6, 5), (7, 5), (8, 5),
              (8, 4), (8, 3), (7, 3), (6, 3)]

qubits = [cirq.GridQubit(*idx) for idx in qubit_locations]
num_qubits = len(qubits)

# prepare our DTC circuit list, with 0 through 100 cycles
num_cycles = 100
circuit_list = time_crystals.symbolic_dtc_circuit_list(qubits=qubits, cycles=num_cycles)

# dir to store data in
base_dir = time_crystals.DEFAULT_BASE_DIR

In [4]:
%%time
# Figure 2d

# options for thermalization constant (g) and number of disorder instances to average over
g_options = [0.6, 0.94]
disorder_instances = 36

# random initial states
initial_states = np.random.choice(2, (disorder_instances, num_qubits))

# random noise (h)
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# random gammas
gammas = -np.random.uniform(0.5*np.pi, 1.5*np.pi, (disorder_instances, num_qubits - 1))

# collect average polarizations by g option
average_polarizations = np.empty((2, num_cycles+1, num_qubits))

# evaluate and yield probabilities and their associated initial states for each of the input options for thermalization (g)
for g_index, g in enumerate(g_options):
    # create task
    dtctask = time_crystals.DTCTask(
        qubits=qubits,
        g=g,
        disorder_instances=disorder_instances,
        initial_states=initial_states,
        local_fields=local_fields,
        gammas=gammas)

    # get absolute value of polarizations, averaged over disorder instances
    disorder_averaged_polarizations = time_crystals.simulate_for_polarizations(dtctask, circuit_list)
    
    # store
    average_polarizations[g_index, :, :] = disorder_averaged_polarizations
    
# save data in json format
filename = f'{base_dir}/2d.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 2min 43s, sys: 80.7 ms, total: 2min 43s
Wall time: 2min 43s


In [5]:
%%time
# Figure 3a

# number of disorder instances to average over
disorder_instances = 24

# random noise (h)
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# prepare random and fixed phis to compare
disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

# prepare 3 initial states to compare
neel_initial_state = np.tile([0,1], num_qubits//2)
polarized_initial_state = np.full(num_qubits, 0)
random_initial_state = [0,0,1,1,1,0,0,0,0,1,0,0,1,1,0,0,1,1,1,1][:num_qubits]
initial_states = [neel_initial_state, polarized_initial_state, random_initial_state]

# collect average polarizations by phi option and initial state
average_polarizations = np.empty((2, len(initial_states), num_cycles+1))

# prepare instances iterator
instances_iterator = itertools.product(enumerate([True, False]), enumerate(initial_states))

for (phi_index, random_phi), (initial_state_index, initial_state) in instances_iterator: 
    # use either random, disordered phis or fixed ones
    phis = disordered_phis if random_phi else fixed_phis
    
    # create task
    dtctask = time_crystals.DTCTask(
        qubits=qubits,
        disorder_instances=disorder_instances,
        initial_state=initial_state,
        local_fields=local_fields,
        phis=phis)

    # get absolute value of polarizations, averaged over disorder instances
    disorder_averaged_polarizations = time_crystals.simulate_for_polarizations(dtctask, circuit_list)

    # average over qubit sites, leaving cycles, and store
    average_polarizations[phi_index, initial_state_index, :] = np.mean(disorder_averaged_polarizations, axis=1)

# save data in json format
filename = f'{base_dir}/3a.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 5min 28s, sys: 188 ms, total: 5min 28s
Wall time: 5min 28s


In [6]:
%%time
# Figure 3b

# instances for disorder and random initial states
disorder_instances = 24
initial_state_instances = 20# is 500 in the paper

# prepare random initial states
initial_states = np.random.choice(2, (initial_state_instances, num_qubits))

# disorder instances h
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# Different phis to compare
disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

# collect average polarizations by phi option and initial state
average_polarizations = np.empty((2, initial_state_instances))

for phi_index, random_phi in enumerate([True, False]): 
    # use either random, disordered phis or fixed ones
    phis = disordered_phis if random_phi else fixed_phis

    for initial_state_index, initial_state in enumerate(initial_states):
        # create task
        dtctask = time_crystals.DTCTask(
            qubits=qubits,
            disorder_instances=disorder_instances,
            initial_state=initial_state,
            local_fields=local_fields,
            phis=phis)

        # get absolute value of polarizations, averaged over disorder instances
        disorder_averaged_polarizations = time_crystals.simulate_for_polarizations(dtctask, circuit_list, autocorrelate=False, take_abs=True)
        
        # average over qubit sites and cycles 30 and 31
        averaged_polarization = np.mean(disorder_averaged_polarizations[30:32, ...])
            
        # store average polarizations for this initial state
        average_polarizations[phi_index, initial_state_index] = averaged_polarization

# save data in json format
filename = f'{base_dir}/3b.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 36min 30s, sys: 1.19 s, total: 36min 32s
Wall time: 36min 32s


In [7]:
%%time
# Figure 3c

# number of disorder instances to average over
disorder_instances = 24

# disorder parameters h and phi
local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))

# expand fixed phis for all instances and qubits
disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

# prepare two initial states to compare
disturb_qubit = 11
polarized_initial_state = [0]*num_qubits
disturbed_polarized_initial_state = list(polarized_initial_state)
disturbed_polarized_initial_state[disturb_qubit] = 1
initial_states = [polarized_initial_state, disturbed_polarized_initial_state]
initial_state_instances = len(initial_states)

# collect average polarizations by phi option
average_polarizations = np.empty((2, initial_state_instances, 31, 4))

# prepare instances iterator
instances_iterator = itertools.product(enumerate([disordered_phis, fixed_phis]), enumerate(initial_states))

for (phi_index, phis), (initial_state_index, initial_state) in instances_iterator: 
    
    # create task
    dtctask = time_crystals.DTCTask(
        qubits=qubits,
        disorder_instances=disorder_instances,
        initial_state=initial_state,
        local_fields=local_fields,
        phis=phis)

    # get polarizations, averaged over disorder instances
    disorder_averaged_polarizations = time_crystals.simulate_for_polarizations(dtctask, circuit_list, autocorrelate=False)

    # limit to cycles 30 through 60, and qubits 11 through 14, and save
    average_polarizations[phi_index, initial_state_index, :, :] = disorder_averaged_polarizations[30:61, 11:15]

# save data in json format
filename = f'{base_dir}/3c.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 3min 41s, sys: 84 ms, total: 3min 41s
Wall time: 3min 41s


In [8]:
%%time
# Figure 3d

# prepare two initial states to compare
disturb_qubit = 11
polarized_initial_state = np.full(num_qubits, 0)
disturbed_polarized_initial_state = polarized_initial_state.copy()
disturbed_polarized_initial_state[disturb_qubit] = 1
initial_states = [polarized_initial_state, disturbed_polarized_initial_state]
initial_state_instances = len(initial_states)

# we have different disorder instances for the two initial states
disorder_instances_options = [64, 81]

# collect polarizations first
average_polarizations = np.empty((2, initial_state_instances, num_cycles + 1, num_qubits))

# iterate over initial states and their associated number of disorder instances
for initial_state_index, (initial_state, disorder_instances) in enumerate(zip(initial_states, disorder_instances_options)): 
    
    # disorder parameters h and phi 
    local_fields = np.random.uniform(-1.0, 1.0, (disorder_instances, num_qubits))
    
    # expand fixed phis for all instances and qubits
    disordered_phis = np.random.uniform(-1.5*np.pi, -0.5*np.pi, (disorder_instances, num_qubits - 1))
    fixed_phis = np.full((disorder_instances, num_qubits - 1), -0.4)

    # expand initial states for each disorder instance
    initial_state_array = np.asarray([initial_state] * disorder_instances)

    average_polarizations_by_phi = []
    for phi_index, phis in enumerate([disordered_phis, fixed_phis]):
        
        # create task
        dtctask = time_crystals.DTCTask(
            qubits=qubits,
            disorder_instances=disorder_instances,
            initial_state=initial_state,
            local_fields=local_fields,
            phis=phis)

        # get polarizations, averaged over disorder instances
        disorder_averaged_polarizations = time_crystals.simulate_for_polarizations(dtctask, circuit_list, autocorrelate=False)

        # store average polarizations
        average_polarizations[phi_index, initial_state_index, :, :] = disorder_averaged_polarizations

        
# save data in json format
filename = f'{base_dir}/3d.json'
with open(filename, 'w+') as f:
    cirq.to_json(average_polarizations, file_or_fn=f)

CPU times: user 10min 49s, sys: 228 ms, total: 10min 49s
Wall time: 10min 49s
